## PyTorch-Tutorial 
## Chapter - 1 (The Classification)
![logo](https://code.kaytouch.biz/wp-content/uploads/2018/05/pytorch-470x250.png)


## Introduction
Pytorch tutorial is a series of tutorials created by me to explain the basic aspects of PyTorch and its implementation. PyTorch is complex to implement but not difficult. If you see it as a way of documentation or documenting a program, then things get much easier to understand. The most interesting part of this series is that I am also a beginner with PyTorch, so what's difficult me I expect to be difficult for five other individuals also, so at least I expect my tutorial could help five other individuals to implement PyTorch.<br>
In this chapter, I have implemented an image classification problem with the help of PyTorch. Here I have explained everything in the most basic way possible so that you could also understand them easily.

## Index
The things that are explained in this classification tutorial are given below.
* Creating a custom dataset
* Creating a neural network in PyTorch
* Training neural network in PyTorch
* Plotting of loss and accuracy curve
* evaluation of performance

## Data-set
Dataset used - [Arthropod Taxonomy Orders Object Detection Dataset](https://www.kaggle.com/mistag/arthropod-taxonomy-orders-object-detection-dataset)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import matplotlib.pyplot as plt
import torch
# Neural networks can be constructed using the torch.nn package.
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# Any results you write to the current directory are saved as output.

ModuleNotFoundError: No module named 'tqdm'

## Device selection
Select your device, whether you to use CPU or GPU for your model.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

In [ ]:
BASE_PATH = '/kaggle/input/arthropod-taxonomy-orders-object-detection-dataset/ArTaxOr/'

In [ ]:
image=[]
labels=[]
for file in os.listdir(BASE_PATH):
    if file=='Coleoptera':
        for c in os.listdir(os.path.join(BASE_PATH, file)):
            if c!='annotations':
                image.append(c)
                labels.append('Coleoptera')
    if file=='Diptera':
        for c in os.listdir(os.path.join(BASE_PATH, file)):
            if c!='annotations':
                image.append(c)
                labels.append('Diptera')
    if file=='Hymenoptera':
        for c in os.listdir(os.path.join(BASE_PATH, file)):
            if c!='annotations':
                image.append(c)
                labels.append('Hymenoptera')
    if file=='Lepidoptera':
        for c in os.listdir(os.path.join(BASE_PATH, file)):
            if c!='annotations':
                image.append(c)
                labels.append('Lepidoptera')
data = {'Images':image, 'labels':labels} 
data = pd.DataFrame(data) 
data.head()

**NOTE** - In the cell above I have created a csv Data-frame from the raw dataset. In your case you might not have to follow this step if you are already provided with csv file which contains the desired input and target value.

In [ ]:
lb = LabelEncoder()
data['encoded_labels'] = lb.fit_transform(data['labels'])
data.head()

## Spliting of Dataset
I have shown two ways to split a dataset into train and validation. One is by splitting it from sratch and another method is by using train_test_split from scikit-learn( the one which I have commented out).


In [ ]:
batch_size = 128
validation_split = .3
shuffle_dataset = True
random_seed= 42

In [ ]:
# Creating data indices for training and validation splits:
# from sklearn.model_selection import train_test_split
# tr, val = train_test_split(data.label, stratify=data.label, test_size=0.1)
dataset_size = len(data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
#train_indices is equivalent to list(tr.index)
#val_indices is equivalent to list(val.index)

In [ ]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

## Transforms
Transforms are common image transformations. They can be chained together using **Compose**.
## Normalization
Normalize a tensor image with mean and standard deviation. Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels, this transform will normalize each channel of the input torch.*Tensor i.e. input[channel] = (input[channel] - mean[channel]) / std[channel]
Convert a PIL Image or numpy.ndarray to tensor.

## ToTensor
Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] if the PIL Image belongs to one of the modes (L, LA, P, I, F, RGB, YCbCr, RGBA, CMYK, 1) or if the numpy.ndarray has dtype = np.uint8

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

 ## Create custom dataset class
 A dataset must contain following functions to be used by data loader later on.

* __init__() function is where the initial logic happens like reading a csv, assigning transforms etc.
* __getitem__() function returns the data and labels. This function is called from dataloader like this:

> img, label = MyCustomDataset.__getitem__(99)  # For 99th item

<br>
An important thing to note is that __getitem__() return a specific type for a single data point (like a tensor, numpy array etc.), otherwise, in the data loader you will get an error like:

> TypeError: batch must contain tensors, numbers, dicts or lists; found 
> class 'PIL.PngImagePlugin.PngImageFile'

<br>
Credits: [PyTorch Custom Dataset Examples](https://github.com/utkuozbulak/pytorch-custom-dataset-examples)

In [ ]:
class Arthopod_Dataset(Dataset):
    def __init__(self, img_data,img_path,transform=None):
        self.img_path = img_path
        self.transform = transform
        self.img_data = img_data
        
    def __len__(self):
        return len(self.img_data)
    
    def __getitem__(self, index):
        img_name = os.path.join(self.img_path,self.img_data.loc[index, 'labels'],
                                self.img_data.loc[index, 'Images'])
        image = Image.open(img_name)
        #image = image.convert('RGB')
        image = image.resize((300,300))
        label = torch.tensor(self.img_data.loc[index, 'encoded_labels'])
        if self.transform is not None:
            image = self.transform(image)
        return image, label

**NOTE** - In case you already have a separate dataset set for train and validation, you could directly pass the path of the datasets to the __init__ section of the custom dataset class and read it right there and use it.

In [ ]:
dataset = Arthopod_Dataset(data,BASE_PATH,transform)

In [ ]:

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [ ]:
def img_display(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    return npimg

## Visualization
visualizing the elements of the dataset

In [ ]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
arthopod_types = {0: 'Coleoptera', 1: 'Diptera', 2: 'Hymenoptera', 3: 'Lepidoptera'}
# Viewing data examples used for training
fig, axis = plt.subplots(3, 5, figsize=(15, 10))
for i, ax in enumerate(axis.flat):
    with torch.no_grad():
        image, label = images[i], labels[i]
        ax.imshow(img_display(image)) # add image
        ax.set(title = f"{arthopod_types[label.item()]}") # add label

## The Neural Network
In the **Net** class created below, we have constructed a neural network. Construction of the neural network was the second most difficult situation that I faced after constructing a custom dataset. But I am going to explain you everything step by step.<br>
* Inside the **init()** method you declare each layer with a unique layer name. For every unique layer, declaring its input features and output features is a must. At least the input feature is a must for some of the layers like batch normalization.
* Inside the **forward(self, x)** method you need to connect the layers that were declared in the init method. One thing must be kept in mind that the output feature of one layer is an input feature of its next connecting layer.



**TIP** - If you are facing difficulty in constructing a neural network class, then I hope this tip would be very much helpful.<br><br>
Before laying your hands on PyTorch I hope you would be familiar with **Keras**( another python library for deep-learning ). And if you are not, then please go practice Keras first as it allows the simplest way of implementing a deep learning model in python.<br><br>
In Keras after implementing a neural network with Model API, there is a command called "Model.summary()" which gives you the entire structure of the neural network that you have created along with the number of parameters. Keep that summary side by side while constructing your network class in PyTorch, things would get much easier. It would be much easier to see that network summary, name the layers according to it and connect them concerning their input and output features in PyTorch. It was helpful in my case, and I hope It would be helpful to you also.
![image](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQylY4AqnlZcpGxTSj7h8tG5ZeUESvLa9MLk51PDDHwkACgirUc)
<br>Link to the model whose summary I used to construct my PyTorch model - [ Classifying Cursive hiragana(崩し字) KMNIST using CNN](https://www.kaggle.com/gpreda/classifying-cursive-hiragana-kmnist-using-cnn)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 3 input image channel, 16 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(3,16,kernel_size=3,stride=2,padding=1)
        self.conv2 = nn.Conv2d(16, 32,kernel_size=3,stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 64,kernel_size=3,stride=2, padding=1)
        self.conv4 = nn.Conv2d(64, 64,kernel_size=3,stride=2, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout2d(0.4)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.batchnorm3 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64*5*5,512 )
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 4)
        
    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.dropout(self.batchnorm2(self.pool(x)))
        x = self.batchnorm3(self.pool(F.relu(self.conv3(x))))
        x = self.dropout(self.conv4(x))
        x = x.view(-1, 64*5*5) # Flatten layer
        x = self.dropout(self.fc1(x))
        x = self.dropout(self.fc2(x))
        x = F.log_softmax(self.fc3(x),dim = 1)
        return x

In [ ]:
model = Net() # On CPU
#model = Net().to(device)  # On GPU
print(model)

## CrossEntropyLoss
It is useful when training a classification problem with C classes. If provided, the optional argument weight should be a 1D Tensor assigning weight to each of the classes. This is particularly useful when you have an unbalanced training set. It is a prototpe of categorical crossentropy in keras.
In case of **Binary classification** use **BCELoss(Binary Cross Entropy)** or BCEWithLogitsLoss.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()


## Training the Network
In the cell below, it is explained how to train your model with epochs. In the "train_loss" and "val_loss" the training loss and validation loss are stored respectively after every epoch. Similarly in case of training accuracy and validation accuracy also, the same thing happens. just remember while validation the weighhts are not upgraded thats why we use **" with torch.no_grad() "**.<br>
Here, **"Torch.max(x, dim=1)"** works same as **"np.argmax(x, axis=1)"**. We use **".item()"** to get the value inside the tensor. **torch.save(model.state_dict(), 'model_classification_tutorial.pt')** is used to save the PyTorch weight in the given directory.

In [ ]:
n_epochs = 12
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader)
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    # scheduler.step(epoch)
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(train_loader):
        #data_, target_ = data_.to(device), target_.to(device)# on GPU
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(data_)
        loss = criterion(outputs, target_)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc: {(100 * correct / total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        model.eval()
        for data_t, target_t in (validation_loader):
            #data_t, target_t = data_t.to(device), target_t.to(device)# on GPU
            outputs_t = model(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t / total_t)
        val_loss.append(batch_loss/len(validation_loader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t / total_t):.4f}\n')
        # Saving the best weight 
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(model.state_dict(), 'model_classification_tutorial.pt')
            print('Detected network improvement, saving current model')
    model.train()

## Accuracy and loss Curve

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.title("Train - Validation Loss")
plt.plot( train_loss, label='train')
plt.plot( val_loss, label='validation')
plt.xlabel('num_epochs', fontsize=12)
plt.ylabel('loss', fontsize=12)
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.title("Train - Validation Accuracy")
plt.plot(train_acc, label='train')
plt.plot(val_acc, label='validation')
plt.xlabel('num_epochs', fontsize=12)
plt.ylabel('accuracy', fontsize=12)
plt.legend(loc='best')

In [ ]:
# Importing trained Network with better loss of validation
model.load_state_dict(torch.load('model_classification_tutorial.pt'))

## Evaluation
evaluating the model performance through visualization

In [ ]:
dataiter = iter(validation_loader)
images, labels = dataiter.next()
arthopod_types = {0: 'Coleoptera', 1: 'Diptera', 2: 'Hymenoptera', 3: 'Lepidoptera'}
# Viewing data examples used for training
fig, axis = plt.subplots(3, 5, figsize=(15, 10))
with torch.no_grad():
    model.eval()
    for ax, image, label in zip(axis.flat,images, labels):
        ax.imshow(img_display(image)) # add image
        image_tensor = image.unsqueeze_(0)
        output_ = model(image_tensor)
        output_ = output_.argmax()
        k = output_.item()==label.item()
        ax.set_title(str(arthopod_types[label.item()])+":" +str(k)) # add label

## What is Pytorch?
### (according to [PyTorch.org](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py))
It’s a Python-based scientific computing package targeted at two sets of audiences:
* A replacement for NumPy to use the power of GPUs
* a deep learning research platform that provides maximum flexibility and speed

### (according to ME 😤)

It is a python based library, useful for a well-documented program in deep learning. I did not observe much speed up in training due to the use of GPU, maybe with multiple GPUs, it might speed up its training. Due to its well-documented feature, I would agree that it is flexible.<br>
I would recommend you to learn PyTorch but after having a hand full of experience in Keras or at least some knowledge about Keras. Pytorch should not be learned right at the get-go.


## Would there be any improvement in model performence while classification by using Pytorch?
As far as I have learned about PyTorch, the answer is no. The same model architecture would give the same performance in Keras. If you find any exception please let me know in the comments down below. I would be happy to learn about it.

## Conclusion
Whatever I have explained in this kernel is according to my personal experience in PyTorch as a beginner. If I have missed out to explain something then please let me know. I would be adding more chapters to this pytorch tutorial series. Till then please **UPVOTE** this kernel if you like it, feel free to let me know what all improvements that could be added to this tutorial and have a nice day.

## Credits
* [PyTorch CNN from scratch](https://www.kaggle.com/bonhart/pytorch-cnn-from-scratch)
* [Starting Kit for PyTorch Deep Learning](https://www.kaggle.com/mratsim/starting-kit-for-pytorch-deep-learning)
* [Simple EDA and model in pytorch](https://www.kaggle.com/artgor/simple-eda-and-model-in-pytorch)
* [CNN - Digit Recognizer (PyTorch)](https://www.kaggle.com/gustafsilva/cnn-digit-recognizer-pytorch)